In [1]:
%pip install beautifulsoup4
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup

# 모바일 지식In > Q&A > 건강 > 건강상담
# https://m.kin.naver.com/mobile/qna/directoryDetail.naver?dirId=7 

# 지식In > Q&A > 건강 > 건강상담
# https://kin.naver.com/qna/expertAnswerList.naver?dirId=701 

# 개별 아티클의 메타 정보
# https://m.kin.naver.com/mobile/qna/kinupList.naver?dirId=701&page=4


# 각 아티클의 url을 받아서 크로울링 해서 question을 반환한다.
def crawl_article_question(article_url):
    raw_html = requests.get(article_url).text
    html = BeautifulSoup(raw_html, 'html.parser')
    question = html.select_one('.user_content').text.strip()
    return question

# 각 아티클의 메타 정보를 받아서 크로울링 하고 데이터를 반환한다.
def crawl_a_data(article_meta_info):

    speciality = article_meta_info["dirName"]
    title = article_meta_info["title"]
    d1Id = article_meta_info["d1Id"]
    dirId = article_meta_info["dirId"]
    docId = article_meta_info["docId"]

    # 글 1개의 url
    article_url = f"https://m.kin.naver.com/mobile/qna/detail.naver?d1Id={d1Id}&dirId={dirId}&docId={docId}"
    # https://m.kin.naver.com/mobile/qna/detail.naver?d1Id=7&dirId=70109&docId=453765047
    question = crawl_article_question(article_url)
    print(article_url)
    print(speciality, d1Id, dirId, docId, f"'{title}'", question)
    a_data = { "title":title, "question":question, "speciality":speciality }
    return a_data


# 크로울링한 전체 데이터가 담길 리스트
datas = []

MAX_PAGE = 10
for page in range(1, MAX_PAGE+1):
    # 아티클 정보 url
    article_info_url = f'https://m.kin.naver.com/mobile/qna/kinupList.naver?dirId=701&page={page}'
    # https://m.kin.naver.com/mobile/qna/kinupList.naver?dirId=701&page=1

    raw_response = requests.get(article_info_url).text
    response = json.loads(raw_response)

    # 1 page에 10개의 아티클 정보를 가지고 있다.
    article_infos = response["lists"]
    for article_info in article_infos:
        datas.append(crawl_a_data(article_info))


https://m.kin.naver.com/mobile/qna/detail.naver?d1Id=7&dirId=70106&docId=453766093
정형외과 7 70106 453766093 '손가락 베였는데 병원 가야할까요' 손가락 베였는데 병원 가야 할 정도인가요? 가면 어디병원을 가여하나요 정형외과를 가야하나요 피부과를 가야하나요?
https://m.kin.naver.com/mobile/qna/detail.naver?d1Id=7&dirId=7011403&docId=453766004
생리, 피임 7 7011403 453766004 '야즈에서 야스민 복용 후 생리??' 제가 부정출혈이 보여서 야즈 복용중이였는데도출혈이 멈추지 않아서 야스민으로 바꿨습니다야즈 하얀알약까지 먹고 야스민으로 바꿨는데하루복용만에 생리처럼 출혈이 계속 나고 있어요 ..지금 4일째 생리하는 것 처럼 나옵니다문제가 있는걸까요?ㅠㅠ
https://m.kin.naver.com/mobile/qna/detail.naver?d1Id=7&dirId=70106&docId=453765961
정형외과 7 70106 453765961 '족저근막염 구두 때문 안낫는건가요?' 안녕하세요 족저근막염에 걸린지 꽤 되었는데요 이게 통증이 점점 심해지는게 너무 힘드네요어느 날은 걷기가 힘들만큼요..제가 치료도 받고 다 해봤는데 너무 안나아요근데 제가 직장 규정상, 구두를 안 신을 수가 없어요이게 치명적인 영향이 있는 건가요?어떻게 해야할까요..
https://m.kin.naver.com/mobile/qna/detail.naver?d1Id=7&dirId=7011403&docId=453765956
생리, 피임 7 7011403 453765956 '생리 1-2일차 관계' 생리 1-2일차에 관계 맺었습니다1일차 콘돔 질외2일차 새벽 콘돔 질외2일차 아침 노콘 배에 사정 후 닦고 샤워그리고 6일차 되는 어제 저녁에 얼리임테기 사용했는데 이런결과가 나왔어요 (5분뒤 촬영)결과가 두줄로 바뀔 가능성이있나요? 두번째 사진은 좀 전에

In [3]:
print(len(datas))

200


In [4]:
import pandas as pd

df = pd.DataFrame(datas)
df.head()

,title,question,speciality
0,손가락 베였는데 병원 가야할까요,손가락 베였는데 병원 가야 할 정도인가요? 가면 어디병원을 가여하나요 정형외과를 가...,정형외과
1,야즈에서 야스민 복용 후 생리??,제가 부정출혈이 보여서 야즈 복용중이였는데도출혈이 멈추지 않아서 야스민으로 바꿨습니...,"생리, 피임"
2,족저근막염 구두 때문 안낫는건가요?,안녕하세요 족저근막염에 걸린지 꽤 되었는데요 이게 통증이 점점 심해지는게 너무 힘드...,정형외과
3,생리 1-2일차 관계,생리 1-2일차에 관계 맺었습니다1일차 콘돔 질외2일차 새벽 콘돔 질외2일차 아침 ...,"생리, 피임"
4,젖은 수건 덮고 자면,젖은 수건 덮고 자면 정말 죽을 수 있나요 평소 잠버릇이 좀 있기는 하지만 수건 쫙...,정신건강의학과


In [5]:
data_file_name = "data/medical_speciality.csv"
df.to_csv(data_file_name, index=False)

In [6]:
import pandas as pd

data_file_name = "data/medical_speciality.csv"
df = pd.read_csv(data_file_name)
df.head()

,title,question,speciality
0,손가락 베였는데 병원 가야할까요,손가락 베였는데 병원 가야 할 정도인가요? 가면 어디병원을 가여하나요 정형외과를 가...,정형외과
1,야즈에서 야스민 복용 후 생리??,제가 부정출혈이 보여서 야즈 복용중이였는데도출혈이 멈추지 않아서 야스민으로 바꿨습니...,"생리, 피임"
2,족저근막염 구두 때문 안낫는건가요?,안녕하세요 족저근막염에 걸린지 꽤 되었는데요 이게 통증이 점점 심해지는게 너무 힘드...,정형외과
3,생리 1-2일차 관계,생리 1-2일차에 관계 맺었습니다1일차 콘돔 질외2일차 새벽 콘돔 질외2일차 아침 ...,"생리, 피임"
4,젖은 수건 덮고 자면,젖은 수건 덮고 자면 정말 죽을 수 있나요 평소 잠버릇이 좀 있기는 하지만 수건 쫙...,정신건강의학과


# openai 라이브러리 설치

In [118]:
%pip install openai==0.28.0

Note: you may need to restart the kernel to use updated packages.


# 필요 라이브러리 설치

In [119]:
%pip install --upgrade tenacity
%pip install --upgrade numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# openai 사용 선언과 키 설정

In [120]:
import openai

openai.api_key = "sk-..."

# 학습 데이터 준비

학습 데이터는 다음과 같은 형태로 준비한다.

```
[
  {'role': 'system', 'content': 'GPT의 역할을 명시한다.'},
  {'role': 'user', 'content': 'GPT에게 전달할 입력 값'},
  {'role': 'assistant', 'content': 'GPT가 입력 데이터를 받아서 출력할 값'}
 ]
```

본 실습에서는 조리법을 입력으로 하여 재료성분을 출력하려 한다. <br>
GPT는 재료 성분을 추출하는 역할을 하고, 입력으로 읽어들인 데이터에서 title과 ingredient 상세로 하고, 출력으로 재료성분 리스트를 출력하게 한다.<br>


실제 학습 데이터는 다음과 같다.

```
[
  {'role': 'system', 'content': 'You are a helpful medical assistant. You will be provided medical question, and tell which speciality the questiong is related.'},

  {'role': 'user', 'content': 'Title: 집에서 돈벌수 있는 알바\n\nQuestion: 돈을 벌고 싶은데 소심해서 쉽게 나가지를 못합니다  방법이 있을까요?  1자존감  2용기  3자신감  생기는 방법과 집에서 할수있는거 있을까요'},

  {'role': 'assistant', 'content': '정신건강의학과'}
 ]
```

In [7]:
system_message = "You are a helpful medical assistant. You will be provided medical question, and tell which speciality the questiong is related."

def create_user_message(title, question):
    return f"""Title: {title}\n\Question: {question}\n\n """

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row['title'], row['question'])
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row['speciality']})

    return {"messages": messages}

pprint(prepare_example_conversation(df.iloc[0]))

{'messages': [{'content': 'You are a helpful medical assistant. You will be '
                          'provided medical question, and tell which '
                          'speciality the questiong is related.',
               'role': 'system'},
              {'content': 'Title: 손가락 베였는데 병원 가야할까요\n'
                          '\\Question: 손가락 베였는데 병원 가야 할 정도인가요? 가면 어디병원을 가여하나요 '
                          '정형외과를 가야하나요 피부과를 가야하나요?\n'
                          '\n'
                          ' ',
               'role': 'user'},
              {'content': '정형외과', 'role': 'assistant'}]}


# 실제 데이터 생성

In [8]:
train_data_count = int(len(datas)*0.8)
valid_data_count = int(len(datas)*0.1)
test_data_count = len(datas) - train_data_count - valid_data_count


In [9]:
training_df = df.loc[0:train_data_count]

training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful medical assistant. You will be provided medical question, and tell which speciality the questiong is related.'}, {'role': 'user', 'content': 'Title: 손가락 베였는데 병원 가야할까요\n\\Question: 손가락 베였는데 병원 가야 할 정도인가요? 가면 어디병원을 가여하나요 정형외과를 가야하나요 피부과를 가야하나요?\n\n '}, {'role': 'assistant', 'content': '정형외과'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful medical assistant. You will be provided medical question, and tell which speciality the questiong is related.'}, {'role': 'user', 'content': 'Title: 야즈에서 야스민 복용 후 생리??\n\\Question: 제가 부정출혈이 보여서 야즈 복용중이였는데도출혈이 멈추지 않아서 야스민으로 바꿨습니다야즈 하얀알약까지 먹고 야스민으로 바꿨는데하루복용만에 생리처럼 출혈이 계속 나고 있어요 ..지금 4일째 생리하는 것 처럼 나옵니다문제가 있는걸까요?ㅠㅠ\n\n '}, {'role': 'assistant', 'content': '생리, 피임'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful medical assistant. You will be provided medical question, and tell which speciality the questiong is related.'}, {'role': 'user', 'content': 'Title: 족저

In [10]:
validation_df = df.loc[train_data_count:train_data_count+valid_data_count]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

# 데이터 파일로 쓰기

실제 학습은 OpenAI에 업로드 하여 이루어 진다. 업로드하도록 파일로 저장한다.

In [11]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

training_file_name = "tmp/medical_speciality_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp/medical_speciality_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)            

# 파일 업로드

In [142]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-K7jSTIZUN0jOOGQ1NUfUj5wJ
Validation file ID: file-eJrV7WGDXWP046OwbdUpXPCz


# 학습

기존 학습된 모델에 추가 학습하는 것을 fine tuning이라 한다.

In [146]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

Job ID: ftjob-0q3cjD3aNt5ijvyB1tRZAcL2
Status: created


# 학습 상태 조회

In [162]:
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])


Job ID: ftjob-0q3cjD3aNt5ijvyB1tRZAcL2
Status: running
Trained Tokens: None


# 학습 진행 내역 조회

In [167]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 48/96: training loss=0.06, validation loss=1.56
Step 49/96: training loss=0.70, validation loss=0.02
Step 50/96: training loss=0.03, validation loss=1.54
Step 51/96: training loss=0.04, validation loss=0.02
Step 52/96: training loss=0.48, validation loss=1.67
Step 53/96: training loss=0.28, validation loss=0.02
Step 54/96: training loss=0.58, validation loss=1.70
Step 55/96: training loss=0.70, validation loss=0.01
Step 56/96: training loss=0.10, validation loss=1.71
Step 57/96: training loss=0.00, validation loss=0.01
Step 58/96: training loss=0.66, validation loss=1.72
Step 59/96: training loss=0.01, validation loss=0.01
Step 60/96: training loss=0.01, validation loss=1.71
Step 61/96: training loss=0.00, validation loss=0.00
Step 62/96: training loss=0.01, validation loss=1.75
Step 63/96: training loss=0.01, validation loss=0.00
Step 64/96: training loss=0.03, validation loss=1.84
Step 65/96: training loss=0.08, validation loss=0.00
Step 66/96: training loss=0.06, validation los

# 학습된 모델 ID 받기

In [168]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:rowan:recipe-ner:7u6fmjmo


# 테스트 데이터 만들기

In [14]:
test_df = df.loc[train_data_count+valid_data_count:]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
test_messages.append({"role": "user", "content": create_user_message(test_row['title'], test_row['question'])})

pprint(test_messages)

[{'content': 'You are a helpful medical assistant. You will be provided '
             'medical question, and tell which speciality the questiong is '
             'related.',
  'role': 'system'},
 {'content': 'Title: 관계 임신가능성\n'
             '\\Question: 어제, 오늘 남친과 관계를 했는데 아랫배가 좀 아팠는데 왜그럴까요ㅠ그리고 생리를 미뤄야해서 '
             '경구피임약을 먹었는데 괜찮을까요??\n'
             '\n'
             ' ',
  'role': 'user'}]


# 호출

gpt-3.5-turboo와 같은 기존 모델이 아닌, 새로 학습한 모델(fine_tuned_model_id)을 주어 호출한다.

In [170]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)

print(response["choices"][0]["message"]["content"])

정신건강의학과
